In [23]:
!pip install llama-index sentence-transformers cohere anthropic voyageai protobuf pypdf

Looking in indexes: https://mirrors.cloud.aliyuncs.com/pypi/simple
DEPRECATION: pytorch-lightning 1.7.7 has a non-standard dependency specifier torch>=1.9.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pytorch-lightning or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 23.3.2 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [24]:
!pip install llama-index

Looking in indexes: https://mirrors.cloud.aliyuncs.com/pypi/simple
DEPRECATION: pytorch-lightning 1.7.7 has a non-standard dependency specifier torch>=1.9.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pytorch-lightning or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 23.3.2 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [25]:
from llama_index.core import SimpleDirectoryReader
from llama_index.core.node_parser import SentenceSplitter

# 加载数据
documents = SimpleDirectoryReader(input_files=["/mnt/workspace/LLaMA-Factory/paper/HLLM.pdf"]).load_data()

# 创建 SentenceSplitter 对象
node_parser = SentenceSplitter(chunk_size=512)

# 解析文档为节点
nodes = node_parser.get_nodes_from_documents(documents)

# 打印结果
for i, node in enumerate(nodes):
    print(f"Node {i+1}: {node.text[:100]}...") 

Node 1: HLLM: Enhancing Sequential Recommendations via Hierarchical Large
Language Models for Item and User ...
Node 2: Extensive experiments demonstrate that
our method effectively leverages the pre-trained capabilities...
Node 3: 1992; Koren, Bell, and V olinsky 2009;
*Equal contribution. †Corresponding author.
Sarwar et al. 200...
Node 4: 2023b). These explorations can be broadly
categorized into three approaches: (1). Utilizing LLMs to
...
Node 5: performance improvements of existing LLM-based methods
over traditional methods are not significant,...
Node 6: To address these challenges, this paper proposes the
Hierarchical Large Language Model (HLLM) archit...
Node 7: This demonstrates that the world
knowledge embedded in LLMs is indeed valuable for rec-
ommendations...
Node 8: Nevertheless, task-specific
fine-tuning with recommendation objectives is essential.
3) HLLM exhibit...
Node 9: 2023) and HSTU (Zhai et al. 2024)
demonstrate that models with parameter counts within hun-
dred

In [26]:
qa_generate_prompt_tmpl = """\
Context information is below.

---------------------
{context_str}
---------------------

Given the context information and not prior knowledge.
generate only questions based on the below query.

You are a Professor. Your task is to setup \
{num_questions_per_chunk} questions for an upcoming \
quiz/examination. The questions should be diverse in nature \
across the document. The questions should not contain options, not start with Q1/ Q2. \
Restrict the questions to the context information provided.\
"""

In [27]:
import torch
import json
import re
from transformers import AutoModelForCausalLM, AutoTokenizer
from tqdm import tqdm
from collections import defaultdict
# 定义模型名称或路径
model_name = "/mnt/workspace/LLaMA-Factory/saves/Qwen2.5-7B-Instruct/lora/sft/checkpoint-336"

# 加载分词器
tokenizer = AutoTokenizer.from_pretrained(model_name)

# 加载模型
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,  # 使用半精度浮点数以节省显存
    device_map="auto"  # 自动分配到可用的设备（如GPU）
)

OSError: We couldn't connect to 'https://huggingface.co' to load this file, couldn't find it in the cached files and it looks like Qwen2.5-7B-Instruct is not the path to a directory containing a file named config.json.
Checkout your internet connection or see how to run the library in offline mode at 'https://huggingface.co/docs/transformers/installation#offline-mode'.

In [ ]:
def generate_questions(context, num_questions=2):
    prompt = qa_generate_prompt_tmpl.format(
        context_str=context, num_questions_per_chunk=num_questions
    )
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_length=512,
            num_return_sequences=num_questions,
            do_sample=True,
            temperature=0.7,
            top_p=0.9
        )
    questions = [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]
    return questions
qa_pairs = []
for node in nodes:
    questions = generate_questions(node.text, num_questions=2)
    for question in questions:
        qa_pairs.append({"context": node.text, "question": question})

# 打印结果
for pair in qa_pairs[:5]:  # 打印前 5 对
    print(f"Context: {pair['context'][:100]}...")
    print(f"Question: {pair['question']}")
    print("-" * 50)

In [ ]:
from llama_index.embeddings.openai import OpenAIEmbedding
#初始化嵌入模型
embed_model = OpenAIEmbedding()
#构建向量索引
from llama_index.core import VectorStoreIndex

vector_index = VectorStoreIndex(nodes, embed_model=embed_model)
vector_retriever = vector_index.as_retriever(similarity_top_k=10)
#自定义检索器
from llama_index.core import BaseRetriever

class CustomRetriever(BaseRetriever):
    def __init__(self, vector_retriever):
        self._vector_retriever = vector_retriever

    def _retrieve(self, query_bundle):
        retrieved_nodes = self._vector_retriever.retrieve(query_bundle)
        # 假设有一个重新排序器
        if reranker is not None:
            retrieved_nodes = reranker.postprocess_nodes(retrieved_nodes, query_bundle)
        else:
            retrieved_nodes = retrieved_nodes[:5]  # 默认取前 5 个
        return retrieved_nodes
#初始化检索器    
custom_retriever = CustomRetriever(vector_retriever)

In [ ]:
#初始化评估器
from llama_index.evaluation import RetrieverEvaluator

retriever_evaluator = RetrieverEvaluator.from_metric_names(
    ["mrr", "hit_rate"], retriever=custom_retriever
)

# 准备评估数据集
class QADataSet:
    def __init__(self, queries, corpus, relevant_docs):
        self.queries = queries
        self.corpus = corpus
        self.relevant_docs = relevant_docs

# 示例数据
queries = {i: pair["question"] for i, pair in enumerate(qa_pairs)}
corpus = {i: pair["context"] for i, pair in enumerate(qa_pairs)}
relevant_docs = {i: [i] for i in range(len(qa_pairs))}  # 假设每个问题只对应一个相关文档

qa_dataset = QADataSet(queries, corpus, relevant_docs)

In [ ]:
#异步评估

import asyncio

eval_results = await retriever_evaluator.aevaluate_dataset(qa_dataset)

# 打印评估结果
for result in eval_results:
    print(f"Query: {result.query}")
    print(f"Hit Rate: {result.metrics['hit_rate']}")
    print(f"MRR: {result.metrics['mrr']}")
    print("-" * 50)